# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 2:50PM,240982,10,0085821026,ISDIN Corporation,Released
1,Jul 1 2022 2:50PM,240982,10,0085821029,ISDIN Corporation,Released
2,Jul 1 2022 2:50PM,240982,10,0085821032,ISDIN Corporation,Released
3,Jul 1 2022 2:50PM,240982,10,0085821028,ISDIN Corporation,Released
4,Jul 1 2022 2:50PM,240982,10,0085821031,ISDIN Corporation,Released
5,Jul 1 2022 2:50PM,240982,10,0085821035,ISDIN Corporation,Released
6,Jul 1 2022 2:50PM,240982,10,0085821033,ISDIN Corporation,Released
7,Jul 1 2022 2:50PM,240982,10,0085821034,ISDIN Corporation,Released
8,Jul 1 2022 2:50PM,240982,10,0085821037,ISDIN Corporation,Released
9,Jul 1 2022 2:50PM,240982,10,0085821036,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,240960,Executing,13
9,240960,Released,3
10,240978,Released,5
11,240981,Released,1
12,240982,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240959,11.0,NaN
240960,13.0,3.0
240978,NaN,5.0
240981,NaN,1.0
240982,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240918,0.0,6.0
240932,0.0,1.0
240941,0.0,1.0
240949,1.0,0.0
240953,0.0,43.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240918,0,6
240932,0,1
240941,0,1
240949,1,0
240953,0,43


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240918,0,6
1,240932,0,1
2,240941,0,1
3,240949,1,0
4,240953,0,43


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240918,,6
1,240932,,1
2,240941,,1
3,240949,1,
4,240953,,43


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation
19,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc."
20,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc."
25,Jul 1 2022 12:34PM,240960,10,Bio-PRF
41,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc."
52,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.
77,Jul 1 2022 11:46AM,240953,10,ISDIN Corporation
120,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc."
121,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC"
122,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation,,19
1,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc.",,1
2,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",,5
3,Jul 1 2022 12:34PM,240960,10,Bio-PRF,13,3
4,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",11,
5,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,11,14
6,Jul 1 2022 11:46AM,240953,10,ISDIN Corporation,,43
7,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1,
8,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",,1
9,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation,19,
1,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc.",1,
2,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
3,Jul 1 2022 12:34PM,240960,10,Bio-PRF,3,13
4,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",,11
5,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,14,11
6,Jul 1 2022 11:46AM,240953,10,ISDIN Corporation,43,
7,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",,1
8,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1,
9,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation,19,
1,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc.",1,
2,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
3,Jul 1 2022 12:34PM,240960,10,Bio-PRF,3,13
4,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",,11


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation,19.0,NaN
1,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc.",1.0,NaN
2,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,NaN
3,Jul 1 2022 12:34PM,240960,10,Bio-PRF,3.0,13.0
4,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",NaN,11.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:50PM,240982,10,ISDIN Corporation,19.0,0.0
1,Jul 1 2022 2:41PM,240981,10,"Citieffe, Inc.",1.0,0.0
2,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,0.0
3,Jul 1 2022 12:34PM,240960,10,Bio-PRF,3.0,13.0
4,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",0.0,11.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1686727,73.0,13.0
16,481836,12.0,0.0
20,481915,14.0,22.0
21,240949,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1686727,73.0,13.0
1,16,481836,12.0,0.0
2,20,481915,14.0,22.0
3,21,240949,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,73.0,13.0
1,16,12.0,0.0
2,20,14.0,22.0
3,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,16,Released,12.0
2,20,Released,14.0
3,21,Released,0.0
4,10,Executing,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,20,21
Status,,,,
Executing,13.0,0.0,22.0,1.0
Released,73.0,12.0,14.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,20,21
0,Executing,13.0,0.0,22.0,1.0
1,Released,73.0,12.0,14.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,20,21
0,Executing,13.0,0.0,22.0,1.0
1,Released,73.0,12.0,14.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()